Generación de resúmenes.  Implementa  un  método  de  resumen  extractivo  de  posts basado  en  frecuencias  y  evalúa  el  resultado.  

Deberás  realizar  distintas  pruebas  para 
demostrar  que  el  método  es  adecuado  para  el  tipo  de  textos,  realizando  los  ajustes 
necesarios  para  su  correcto  funcionamiento.  

La  función  se  denominará 
post_summarisation(text: str)

### Método de resumen extractivo basado en frecuencias



Beneficios de usar el texto original ('post') a la hora de hacer el resumen:

- Preserva la forma exacta del texto

- En un resumen extractivo, normalmente se desea mantener intactas las frases que se toman del original para no alterar su significado literal ni su estilo.

- Al no lematizar, no se cambian conjugaciones ni palabras, lo que garantiza que el texto sea una cita fiel del documento fuente.

- Mayor riqueza lingüística y matices

- Las diferencias gramaticales (tiempo verbal, género, número) pueden aportar matices que el lector valora.

- Menor riesgo de “romper” la coherencia en frases

- Si en un resumen extractivo seleccionamos oraciones tal cual, no necesitamos reconstruir sintácticamente nada.

- Dejar las palabras como están minimiza problemas de concordancia (por ejemplo, si “el niño comía” se lematiza y luego sufre transformaciones, podríamos terminar con “el niño comer”, perdiendo coherencia).

- Tokenizar (sin lematizar) es más sencillo y rápido; se reduce la complejidad y el coste computacional de la etapa de preprocesamiento.


In [13]:
import os 
if 'nltk.py' in os.listdir('.') or 'nltk.ipynb' in os.listdir('.'):
    print('hey')

In [1]:
import nltk

# Descargar los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Currito\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Currito\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Currito\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
def post_summarisation_full_text(text):
    from nltk.corpus import stopwords
    from nltk.tokenize import sent_tokenize, word_tokenize
    import string
    """
    Genera un resumen extractivo usando frecuencia de palabras.
    text: Texto original (multi-línea) del que se extraerá el resumen.
    summary_sentences: Número de oraciones que quieres en tu resumen.
    """
    # Separar en oraciones
    sentences = sent_tokenize(text)
    
    # Separar en palabras cada oración para calcular frecuencia
    stop_words = set(stopwords.words('english'))  # Ajusta idioma si es necesario
    word_frequencies = {}
    
    for sentence in sentences:
        # Tokenizar en palabras y limpiar signos de puntuación
        words = word_tokenize(sentence)
        for word in words:
            # Conviertes todo a minúscula y quitas signos de puntuación
            word = word.lower()
            if word not in stop_words and word not in string.punctuation:
                word_frequencies[word] = word_frequencies.get(word, 0) + 1
    
    # Calcular la frecuencia máxima para normalizar
    max_frequency = max(word_frequencies.values()) if word_frequencies else 1
    
    # Asignar puntuación a cada oración según la frecuencia de sus palabras
    sentence_scores = {}
    for sentence in sentences:
        sentence_scores[sentence] = 0
        words_in_sentence = word_tokenize(sentence.lower())
        for word in words_in_sentence:
            if word in word_frequencies:
                # Se suma la frecuencia normalizada de la palabra
                sentence_scores[sentence] += word_frequencies[word] / max_frequency

    # Ordenar oraciones según la puntuación y escoger las mejores
    ranked_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    final_sentences = len(sentences) // 2
    summary = " ".join(ranked_sentences[:final_sentences])
    return summary

text_example = """Aprender a programar requiere práctica y dedicación. Python es un 
    excelente lenguaje para comenzar debido a su sintaxis sencilla y 
    versatilidad. Además, la comunidad de Python es muy activa y proporciona 
    numerosos recursos para desarrolladores de todos los niveles."""

resumen = post_summarisation_full_text(text_example)
print(f"Resumen extractivo:")
print(resumen)

Resumen extractivo:
Además, la comunidad de Python es muy activa y proporciona 
    numerosos recursos para desarrolladores de todos los niveles.


### Resumen con tokens de la columna 'clean_post' en lugar de 'post'.

En esta función tomamos la 'string' de tokens de la columna, calculamos las frecuencias y ordenamos para quedarnos con el 50% de aquellos
más comunes. Perdemos legibilidad pero podemos hacernos una mejor idea del tema del texto. 

```python 
    # Calcular frecuencias
    word_frequencies = {}
    for token in token_list:
        word_frequencies[token] = word_frequencies.get(token, 0) + 1
    
    # Ordenar tokens por frecuencia
    sorted_tokens = sorted(word_frequencies.items(), 
                         key=lambda x: x[1], 
                         reverse=True)

In [18]:
def token_based_post_summarisation(tokens: str) -> str:
    """
    Genera un resumen usando el 25% de los tokens más relevantes.
    
    Args:
        tokens (str): String de tokens preprocesados separados por espacios
    
    Returns:
        str: Resumen con los tokens más frecuentes
    """
    # Convertir string de tokens a lista
    token_list = tokens.split()
    
    # Calcular frecuencias
    word_frequencies = {}
    for token in token_list:
        word_frequencies[token] = word_frequencies.get(token, 0) + 1
    
    # Ordenar tokens por frecuencia
    sorted_tokens = sorted(word_frequencies.items(), 
                         key=lambda x: x[1], 
                         reverse=True)
    
    # Seleccionar el 50% de los tokens más frecuentes
    num_tokens = max(1, len(token_list) // 2)
    selected_tokens = [token for token, _ in sorted_tokens[:num_tokens]]
    
    return " ".join(selected_tokens)

# Ejemplo de uso
clean_tokens_example = "learn program require practice dedication python excellent language \
    start due simple syntax versatility community python active provide numerous resource developer \
        level beginner find tutorial detailed documentation library python cover practically programming need"


print("\nResumen (50% de los tokens más frecuentes):")
print(token_based_post_summarisation(clean_tokens_example))


Resumen (50% de los tokens más frecuentes):
python learn program require practice dedication excellent language start due simple syntax versatility community active provide
